In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
import scipy.io as sp
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans

dataset = sp.loadmat('breastw.mat')

X = dataset['X']
y = dataset['y']

skf= StratifiedKFold(n_splits=10,shuffle=False,random_state=None)
 
test=False
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    LOF = LocalOutlierFactor(n_neighbors=round(len(X_test)*0.6), algorithm='auto', contamination=0.357, metric="manhattan")
    y_pred_LOF = LOF.fit_predict(X_test)
    y_pred_LOF[y_pred_LOF>0]=0
    y_pred_LOF[y_pred_LOF<0]=1
    
    model = IsolationForest(contamination=0.4, n_estimators=30, max_samples=100,max_features=7)
    model.fit(X_train)
    y_pred_iso=model.predict(X_test)
    y_pred_iso[y_pred_iso>0]=0
    y_pred_iso[y_pred_iso<0]=1
    
    kmeans = KMeans(n_clusters=2, init='k-means++' ,n_init=1 ,max_iter=120 ,algorithm='auto').fit(X_test)
    y_pred_kmeans=kmeans.labels_
    
    F1_LOF=round(f1_score(y_test,y_pred_LOF),3)
    F1_iso=round(f1_score(y_test,y_pred_iso),3)
    F1_kmeans=round(f1_score(y_test,y_pred_kmeans),3)
    
    if F1_kmeans < 0.1 :
        F1_kmeans=1-F1_kmeans
    if test == False :
        matrice_LOF = np.array([[F1_LOF]])
        matrice_iso = np.array([[F1_iso]])
        matrice_kmeans = np.array([[F1_kmeans]])
        test = True
    else :
        matrice_LOF = np.append(matrice_LOF,np.array([F1_LOF]).reshape(1,1),axis=0)
        matrice_iso = np.append(matrice_iso,np.array([F1_iso]).reshape(1,1),axis=0)
        matrice_kmeans = np.append(matrice_kmeans,np.array([F1_kmeans]).reshape(1,1),axis=0)

df_LOF=pd.DataFrame(matrice_LOF,columns=["LOF"])
df_iso=pd.DataFrame(matrice_iso,columns=["Isolation Forest"])
df_kmeans=pd.DataFrame(matrice_kmeans,columns=["Kmeans"])
Comparaison=pd.concat([df_LOF, df_iso, df_kmeans], axis = 1)
display(Comparaison)

FileNotFoundError: [Errno 2] No such file or directory: 'breastw.mat'